In [171]:
import numpy as np

In [172]:
def generateData(d, N, noise=0.1):
    np.random.seed(12345)
    X = np.concatenate([np.ones([N, 1]), np.random.randn(N, d)], axis=1)
    lambdaTrue = -1 + 2 * np.tan(np.random.randn(len(X[0])))
    Y = 1 * ((X @ lambdaTrue + noise * np.random.randn(N)) > 0)
    D = X, Y
    return D

In [173]:
def split(D, train_size):
    x, y = D
    x_trainning = []
    x_testing = []
    y_traning = []
    y_testing = []
    for i in range(0, train_size):
        x_trainning.append(x[i])
        y_traning.append(y[i])
    for i in range(train_size, len(y)):
        x_testing.append(x[i])
        y_testing.append(y[i])
    return x_trainning, x_testing, y_traning, y_testing

In [174]:
def model(x, par):
    z = x @ par
    # return 1 / (np.exp(-z)+1)
    return np.power(1 / (1 + np.power(np.e, -z)), -1)

In [175]:
def dModel(x, par):
    f = np.power((1 + np.power(np.e, (np.dot(x, par)))), -1)
    gradient = np.dot(f * (1 - f), x)
    return gradient

In [176]:
def objective(par, data):
    ell = 0
    X, Y = data
    for n in range(0, len(X)):
        x, y = X[n], Y[n]
        # f = np.power((np.dot(par,x)),np.e)
        f = model(X, par)
        if y == 1:
            f = (f)
        else:
            deltat = (1 - f)
        s = -(np.log(f))
        ell = ell + s
    return ell

In [177]:
def gradient(par, data):
    grad = np.zeros(len(par))
    X, Y = data
    for n in range(0, len(X)):
        f = model(X[n], par)
        x, y = X[n], Y[n]
        # deltaf = (dModel(X,par))
        s = -(y - f) * (dModel(x, par)) / (y * f + (1 - y) * (1 - f))  #not sure if deltaf should be here
        # s = -(y-f)/(y*f+(1-y)*(1-f))
        grad = grad + s
    return grad

In [178]:
def train(par0, eta, T, data):
    par = par0
    obj = []
    for t in range(0, T):
        ell = objective(par, data)
        obj.append(ell)
        grad = gradient(par, data)
        par = par - grad * eta
    return par, obj

In [179]:
lambda1 = np.transpose(np.full(11, 0.01))
d = 10
N = 100
D = generateData(d, N)
x_trainning, x_testing, y_traning, y_testing = split(D, 50)
eta = 0.01
epoch = []
itterations = 5
for i in range(0, itterations):
    par, values = train(np.random.normal(0, 1, len(x_trainning[0])), eta, itterations, (x_trainning, y_traning))
    epoch.append((par, values))

In [180]:
def ER(par, data):
    ER = 0
    X, Y = data
    for n in range(0, len(D) - 1):
        x, y = X[n], Y[n]
        yHat = model(x, par[n])
        s = 1 * (y != yHat)
        ER = ER + s
    return ER / len(data)

In [181]:
ERs = []
for i in epoch:
    ERs.append(ER(i, (x_testing, y_testing)))
print(ERs)

[0.5, 0.5, 0.5, 0.5, 0.5]
